In [1]:
# SH-I

In [2]:
import pandas as pd
import numpy as np

In [3]:
lectins = ['ACG', 'CTB', 'DC_SIGN', 'Gal4', 'LCA', 'MAL', 'NPL', 'PSA', 'SBA', 'SNL', 'UEA']

In [4]:
def align(lectin, save = False):

    cfg = pd.read_excel('figures/cfg/CFG_' + lectin + '.xls')
    liga = pd.read_csv('figures/liga/LiGA_' + lectin + '.csv')
    liga_sd = pd.read_csv('figures/liga/StDev/' + lectin + '_vs_YG.csv')

    order = pd.read_csv('figures/order.csv')

    cfg_glycans = pd.read_csv('figures/cfg_glycans.csv')
    liga_glycans = pd.read_csv('figures/liga_glycans.csv')

    liga_sd_dict = dict(zip(liga_sd['Mod'], liga_sd['totalStd']))
    liga['LiGA_StDev'] = liga['Mod'].map(liga_sd_dict).fillna(0)

    assert len(cfg['Structure on Masterlist']) == 611, 'Not the same length bruh.'
    cfg['IUPAC (CFG)'] = cfg_glycans['IUPAC'].values
    liga_dict = dict(zip(liga_glycans['Glycan'], liga_glycans['IUPAC']))
    liga['IUPAC (LiGA)'] = liga['Mod'].map(liga_dict)
    cfg = cfg[['IUPAC (CFG)', 'Average RFU', 'StDev']]
    liga = liga[['IUPAC (LiGA)', 'total', 'LiGA_StDev']]

    label(liga, 'IUPAC (LiGA)')
    label(cfg, 'IUPAC (CFG)')
    label(order, 'IUPAC (LiGA)')
    label(order, 'IUPAC (CFG)')

    cfg_val_dict = dict(zip(cfg['IUPAC (CFG)'], cfg['Average RFU']))
    cfg_std_dict = dict(zip(cfg['IUPAC (CFG)'], cfg['StDev']))
    liga_val_dict = dict(zip(liga['IUPAC (LiGA)'], liga['total']))
    liga_std_dict = dict(zip(liga['IUPAC (LiGA)'], liga['LiGA_StDev']))

    order['total'] = order['IUPAC (LiGA)'].map(liga_val_dict)
    order['LiGA_StDev'] = order['IUPAC (LiGA)'].map(liga_std_dict)
    order['Average RFU'] = order['IUPAC (CFG)'].map(cfg_val_dict)
    order['StDev'] = order['IUPAC (CFG)'].map(cfg_std_dict)

    aligned = order[['IUPAC (LiGA)', 'total', 'LiGA_StDev', 'IUPAC (CFG)', 'Average RFU', 'StDev']]
    aligned.iloc[-1, 3:] = aligned.iloc[-1, 3:].fillna(0)

    unlabel(aligned, 'IUPAC (LiGA)')
    unlabel(aligned, 'IUPAC (CFG)')
    
    if save:
        
        aligned.to_csv('figures/outputs/' + lectin + '.csv', index = False)
    
    return aligned

In [5]:
def top(lectin, save = False):
    
    cfg = pd.read_excel('figures/cfg/CFG_' + lectin + '.xls')
    cfg_glycans = pd.read_csv('figures/cfg_glycans.csv')

    assert len(cfg['Structure on Masterlist']) == 611, 'Not the same length bruh.'
    cfg['IUPAC (CFG)'] = cfg_glycans['IUPAC'].values
    cfg = cfg.sort_values('Average RFU', ascending = False)

    cfg['Norm'] = norm(cfg['Average RFU'].values)

    top = cfg[cfg['Norm'] > 0.2]
    top = top[['IUPAC (CFG)', 'Average RFU', 'StDev']]
    top = top.sort_index()
    top = top.reset_index(drop = True)
    
    if save:
        
        top.to_csv('figures/outputs/' + lectin + '_top.csv', index = False)
    
    return top

In [6]:
def norm(arr):
    
    p = (arr - arr.min())
    q = arr.max() - arr.min()
    
    return p / q

def label(df, col):
    
    df[col] = df[col] + '_' + df.groupby([col]).cumcount().astype(str)
    
def unlabel(df, col):
    
    df[col] = [str(x).split('_')[0] for x in df[col]]
    df[col] = df[col].replace('nan', np.NaN)

In [7]:
for lectin in lectins:

    align(lectin, save = True)
    top(lectin, save = True)

WARNING *** file size (272385) not 512 + multiple of sector size (512)
WARNING *** file size (272385) not 512 + multiple of sector size (512)
